In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml_tensorflow import GroceryML
from grocery_ml_core import GroceryMLCore
from hidden_layer_param_builder import HiddenLayerParamSetBuilder
import tensorflow as tf
import logging

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

logging.basicConfig(level=logging.INFO)

try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    # groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


INFO:ItemIdMapper:ItemIdMapper initialized


C:\Users\steve\source\repos\grocery-ml


INFO:ItemNameUtils:canonicalize_items(): start canonical='milk' patterns=5
INFO:ItemNameUtils:canonicalize_items(): done canonical='milk' total_replaced=14
INFO:ItemNameUtils:canonicalize_items(): start canonical='bread' patterns=7
INFO:ItemNameUtils:canonicalize_items(): done canonical='bread' total_replaced=20
INFO:ItemNameUtils:canonicalize_items(): start canonical='bread' patterns=1
INFO:ItemNameUtils:canonicalize_items(): done canonical='bread' total_replaced=1
INFO:ItemNameUtils:canonicalize_items(): start canonical='icecream' patterns=3
INFO:ItemNameUtils:canonicalize_items(): done canonical='icecream' total_replaced=8
INFO:ItemNameUtils:canonicalize_items(): start canonical='cheese' patterns=4
INFO:ItemNameUtils:canonicalize_items(): done canonical='cheese' total_replaced=18
INFO:ItemNameUtils:canonicalize_items(): start canonical='mayo' patterns=2
INFO:ItemNameUtils:canonicalize_items(): done canonical='mayo' total_replaced=2
INFO:ItemNameUtils:canonicalize_items(): start cano

Building Training DF: Start
_build_combined_df()
_build_sources()


INFO:ItemNameUtils:canonicalize_items(): done canonical='junk-food' total_replaced=34
INFO:ItemNameUtils:canonicalize_items(): start canonical='cereal-raisn-bran-apl-jck_cano' patterns=4
INFO:ItemNameUtils:canonicalize_items(): done canonical='cereal-raisn-bran-apl-jck_cano' total_replaced=23
INFO:ItemNameUtils:canonicalize_items(): start canonical='minute-maid-drink' patterns=3
INFO:ItemNameUtils:canonicalize_items(): done canonical='minute-maid-drink' total_replaced=12
INFO:ItemNameUtils:canonicalize_items(): start canonical='eggs' patterns=3
INFO:ItemNameUtils:canonicalize_items(): done canonical='eggs' total_replaced=11
INFO:ItemNameUtils:canonicalize_items(): start canonical='sparkling-ice' patterns=4
INFO:ItemNameUtils:canonicalize_items(): done canonical='sparkling-ice' total_replaced=78
INFO:ItemNameUtils:canonicalize_items(): start canonical='drinking-water' patterns=2
INFO:ItemNameUtils:canonicalize_items(): done canonical='drinking-water' total_replaced=4
INFO:ItemNameUtils:

creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
build_trip_interveral_feautres(): start
build_trip_interveral_feautres(): done
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
Building Training DF: Done


In [2]:
import optuna 
def new_objective(trial):

    lr = trial.suggest_float("learning_rate", 0.0001, 0.001)
    depth = trial.suggest_int("depth", 3, 20, step=5)
    base_units = trial.suggest_int("base_units", 8, 2048, step=12)
    decay = trial.suggest_float("decay", 0.2, 0.6)
    embedding_dim = trial.suggest_int("embedding_dim", 20, 300, step=10)
    epochs = trial.suggest_int("epochs", 30, 200, step=10)
    output_activation = "sigmoid"
    metrics = ["AUC", "Precision", "Recall"]
    
    layers_cfg = []
    current_units = base_units
    
    for i in range(depth):
        layers_cfg.append({
            "units": int(current_units),
            "activation": "relu"
        })
        current_units = max(4, current_units * decay)

    build_params = {
        "embedding_dim": embedding_dim,
        "layers": layers_cfg,
        "output_activation": output_activation,
        "optimizer": "adam",
        "learning_rate": lr,
        "loss": "binary_crossentropy" if output_activation == "sigmoid" else "mse",
        "metrics": metrics
    }

    train_params = { "epochs": epochs, "batch_size": 32 }
    days = 10
    start_date = pd.Timestamp.now()
    outputDir = f"f:/exp/keras/optuna/{trial.study.study_name}/trial-{trial.number}"
    groceryML.run_experiment_with_consecutive_predictions( groceryML.training_df, build_params, train_params,  outputDir, start_date,  days)                     
    # groceryML.run_experiment(groceryML.training_df, build_params, train_params, outputDir)

    return groceryML.last_val_auc
############################################################################


In [3]:
sampler = optuna.samplers.TPESampler()

study_name = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"
study = optuna.create_study(
    study_name= study_name,
    sampler=sampler,
    direction="maximize",   # or minimize — see note below
    storage="sqlite:///optuna_grocery.db",
    load_if_exists=True
)
study.optimize(new_objective, n_trials=200)

[I 2026-01-22 23:25:22,164] A new study created in RDB with name: 20260122_232521
C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\optuna\distributions.py:684: UserWarning: The distribution is specified by [3, 20] and step=5, but the range is not divisible by `step`. It will be replaced with [3, 18].
  warnings.warn(


Creating dir: f:/exp/keras/optuna/20260122_232521/trial-0\e60_l668-372-207_ep120_sig_523
run_experiment_with_consecutive_predictions() exp_dir: f:/exp/keras/optuna/20260122_232521/trial-0\e60_l668-372-207_ep120_sig_523
when: 2026-01-22 23:25:22.227853 params: {'epochs': 120, 'batch_size': 32}
normalize_features()
train_model()
build_prediction_input() prediction_date=2026-01-22 23:25:22.226852
Building Live DF: Start
_build_combined_df()
_build_sources()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:335: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])
INFO:ItemNameUtils:canonicalize_items(): start canonical='milk' patterns=5
INFO:ItemNameUtils:canonicalize_items(): done canonical='milk' total_replaced=14
INFO:ItemNameUtils:canonicalize_items(): start canonical='bread' patterns=7
INFO:ItemNameUtils:canonicalize_items(): done canonical='bread' total_replaced=22
INFO:ItemNameUtils:canonicalize_items(): start canonical='bread' patterns=1
INFO:ItemNameUtils:canonicalize_items(): done canonical='bread' total_replaced=1
INFO:ItemNameUtils:canonicalize_items(): start canonical='icecream' patterns=3
INFO:ItemNameUtils:canonicalize_items(): done canonical='icecream' total_replaced=9
INFO:ItemNameUtils:canonicalize_items(): st

creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
build_trip_interveral_feautres(): start
build_trip_interveral_feautres(): done
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
Building Live DF: Done
_build_latest_rows_df: start
_build_latest_rows_df: done
create_item_supply_level_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
normalize_features()
18/18 [==============================] - 0s 995us/step


INFO:ItemIdMapper:map_item_ids_to_names(): start rows=572 col_name='item'
INFO:ItemIdMapper:map_item_ids_to_names(): done
C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:335: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])
INFO:ItemNameUtils:canonicalize_items(): start canonical='milk' patterns=5
INFO:ItemNameUtils:canonicalize_items(): done canonical='milk' total_replaced=14
INFO:ItemNameUtils:canonicalize_items(): start canonical='bread' patterns=7
INFO:ItemNameUtils:canonicalize_items(): done canonical='bread' total_replaced=22
INFO:ItemNameUtils:canonicalize_items(): start canonical='bread' patterns=1
INFO:ItemNameUtils:canonicalize_items(): done canonical='bread' total_replaced=1
INFO:ItemNameUtils:canonicalize_items(): start canonical='icecream' patterns=3
INFO:

...

build_prediction_input() prediction_date=2026-01-28 00:33:26.332125
Building Live DF: Start
_build_combined_df()
_build_sources()


INFO:ItemNameUtils:canonicalize_items(): done canonical='steak-ribs-pork-ground-beef-cano' total_replaced=59
INFO:ItemNameUtils:canonicalize_items(): start canonical='frozen-breakfast-jimmy-dean-cano' patterns=1
INFO:ItemNameUtils:canonicalize_items(): done canonical='frozen-breakfast-jimmy-dean-cano' total_replaced=6
INFO:ItemNameUtils:canonicalize_items(): start canonical='shampoo-conditioner-cano' patterns=2
INFO:ItemNameUtils:canonicalize_items(): done canonical='shampoo-conditioner-cano' total_replaced=11
INFO:ItemNameUtils:canonicalize_items(): start canonical='soap' patterns=1
INFO:ItemNameUtils:canonicalize_items(): done canonical='soap' total_replaced=8
INFO:ItemNameUtils:canonicalize_items(): start canonical='yogurt' patterns=3
INFO:ItemNameUtils:canonicalize_items(): done canonical='yogurt' total_replaced=31
INFO:ItemNameUtils:canonicalize_items(): start canonical='coke' patterns=5
INFO:ItemNameUtils:canonicalize_items(): done canonical='coke' total_replaced=135
INFO:ItemNam

creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()


[W 2026-01-23 00:48:17,324] Trial 3 failed with parameters: {'learning_rate': 0.0006480588093207937, 'depth': 13, 'base_units': 1472, 'decay': 0.3775446956865567, 'embedding_dim': 140, 'epochs': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\steve\AppData\Local\Temp\ipykernel_26324\1510849792.py", line 37, in new_objective
    groceryML.run_experiment_with_consecutive_predictions( groceryML.training_df, build_params, train_params,  outputDir, start_date,  days)
  File "C:\Users\steve\source\repos\grocery-ml\grocery_ml_tensorflow.py", line 449, in run_experiment_with_consecutive_predictions
    artifacts = self.build_prediction_input(prediction_date, norm_params)
  File "C:\Users\steve\source\repos\grocery-ml\grocery_ml_tensorflow.py", line 520, in build_prediction_inp

KeyboardInterrupt: 